In [1]:
import torch
import datasets
import os
import aqlm
import time
import numpy as np

In [2]:
import sys
sys.path.append("d:\\projects\\affect\\TUCORE-GCN\\tucore_gcn_transformers")
sys.path.append("d:\\projects\\affect\\TUCORE-GCN\\")
sys.path

['d:\\projects\\affect\\TUCORE-GCN',
 'C:\\Users\\picokatx\\AppData\\Local\\Programs\\Python\\Python311\\python311.zip',
 'C:\\Users\\picokatx\\AppData\\Local\\Programs\\Python\\Python311\\DLLs',
 'C:\\Users\\picokatx\\AppData\\Local\\Programs\\Python\\Python311\\Lib',
 'C:\\Users\\picokatx\\AppData\\Local\\Programs\\Python\\Python311',
 'd:\\projects\\affect\\TUCORE-GCN\\.venv',
 '',
 'd:\\projects\\affect\\TUCORE-GCN\\.venv\\Lib\\site-packages',
 'D:\\projects\\affect\\TUCORE-GCN\\AQLM\\src',
 'd:\\projects\\affect\\TUCORE-GCN\\.venv\\Lib\\site-packages\\win32',
 'd:\\projects\\affect\\TUCORE-GCN\\.venv\\Lib\\site-packages\\win32\\lib',
 'd:\\projects\\affect\\TUCORE-GCN\\.venv\\Lib\\site-packages\\Pythonwin',
 'd:\\projects\\affect\\TUCORE-GCN\\tucore_gcn_transformers',
 'd:\\projects\\affect\\TUCORE-GCN\\']

In [6]:
torch.tensor(0).to("cuda")

: 

In [4]:
from berttest import create_block, create_mixer_cls, create_mlp_cls
from transformers.models.bert.modeling_bert import BertModel, BertConfig
config = BertConfig.from_json_file("./models/BERT/tucoregcn_bert_mlc.json")
state_dict = create_block(config, 0).to(dtype=torch.float16).state_dict()

In [5]:
state_dict['mixer.Wqkv.bias']

tensor([-0.0114,  0.0061, -0.0095,  ...,  0.0163, -0.0275,  0.0054],
       dtype=torch.float16)

In [3]:
from transformers.models.bert.modeling_bert import BertModel, BertConfig
from berttest import BertModel as FlashBertModel
config = BertConfig.from_json_file("./models/BERT/tucoregcn_bert_mlc.json")
fbert = FlashBertModel(config=config)
bertmodel = BertModel.from_pretrained("bert-base-uncased", config=config)
bertmodel

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [4]:
fbert

OrderedDict([('embeddings.word_embeddings.weight',
              tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
                      [-0.0409, -0.0223,  0.0249,  ..., -0.0077, -0.0179, -0.0006],
                      [ 0.0178, -0.0211, -0.0404,  ...,  0.0156,  0.0167,  0.0296],
                      ...,
                      [-0.0518,  0.0155, -0.0168,  ...,  0.0013,  0.0010,  0.0112],
                      [ 0.0256,  0.0171, -0.0024,  ..., -0.0400,  0.0209,  0.0630],
                      [ 0.0086, -0.0098, -0.0092,  ...,  0.0108, -0.0296, -0.0363]])),
             ('embeddings.position_embeddings.weight',
              tensor([[ 8.7329e-04,  2.8825e-02,  7.5620e-03,  ...,  1.7959e-02,
                       -2.6846e-02,  4.8092e-03],
                      [ 1.4655e-02, -2.9485e-02, -1.1262e-04,  ..., -2.5672e-03,
                        3.2528e-02,  2.6433e-02],
                      [ 7.3537e-04,  5.5280e-03, -6.1467e-03,  ...,  9.5459e-06,
                 

In [4]:
from transformers import PretrainedConfig
import re
from collections import OrderedDict
from collections.abc import Sequence
from functools import partial
from typing import Any, List, Mapping, Optional, Tuple, Union
import torch.nn.functional as F
def remap_state_dict(state_dict, config: PretrainedConfig):
    """
    Map the state_dict of a Huggingface BERT model to be flash_attn compatible.
    """

    # LayerNorm
    def key_mapping_ln_gamma_beta(key):
        key = re.sub(r"LayerNorm.gamma$", "LayerNorm.weight", key)
        key = re.sub(r"LayerNorm.beta$", "LayerNorm.bias", key)
        return key

    state_dict = OrderedDict((key_mapping_ln_gamma_beta(k), v) for k, v in state_dict.items())

    # Layers
    def key_mapping_layers(key):
        return re.sub(r"^encoder.layer.", "encoder.layers.", key)

    state_dict = OrderedDict((key_mapping_layers(k), v) for k, v in state_dict.items())

    # LayerNorm
    def key_mapping_ln(key):
        key = re.sub(r"^embeddings.LayerNorm.", "emb_ln.", key)
        key = re.sub(
            r"^encoder.layers.(\d+).attention.output.LayerNorm.(weight|bias)",
            r"encoder.layers.\1.norm1.\2",
            key,
        )
        key = re.sub(
            r"^encoder.layers.(\d+).output.LayerNorm.(weight|bias)",
            r"encoder.layers.\1.norm2.\2",
            key,
        )
        key = re.sub(
            r"^predictions.transform.LayerNorm.(weight|bias)",
            r"predictions.transform.layer_norm.\1",
            key,
        )
        return key

    state_dict = OrderedDict((key_mapping_ln(k), v) for k, v in state_dict.items())

    # MLP
    def key_mapping_mlp(key):
        key = re.sub(
            r"^encoder.layers.(\d+).intermediate.dense.(weight|bias)",
            r"encoder.layers.\1.mlp.fc1.\2",
            key,
        )
        key = re.sub(
            r"^encoder.layers.(\d+).output.dense.(weight|bias)",
            r"encoder.layers.\1.mlp.fc2.\2",
            key,
        )
        return key

    state_dict = OrderedDict((key_mapping_mlp(k), v) for k, v in state_dict.items())

    # Attention
    last_layer_subset = getattr(config, "last_layer_subset", False)
    for d in range(config.num_hidden_layers):
        Wq = state_dict.pop(f"encoder.layers.{d}.attention.self.query.weight")
        Wk = state_dict.pop(f"encoder.layers.{d}.attention.self.key.weight")
        Wv = state_dict.pop(f"encoder.layers.{d}.attention.self.value.weight")
        bq = state_dict.pop(f"encoder.layers.{d}.attention.self.query.bias")
        bk = state_dict.pop(f"encoder.layers.{d}.attention.self.key.bias")
        bv = state_dict.pop(f"encoder.layers.{d}.attention.self.value.bias")
        if not (last_layer_subset and d == config.num_hidden_layers - 1):
            state_dict[f"encoder.layers.{d}.mixer.Wqkv.weight"] = torch.cat(
                [Wq, Wk, Wv], dim=0
            )
            state_dict[f"encoder.layers.{d}.mixer.Wqkv.bias"] = torch.cat([bq, bk, bv], dim=0)
        else:
            state_dict[f"encoder.layers.{d}.mixer.Wq.weight"] = Wq
            state_dict[f"encoder.layers.{d}.mixer.Wkv.weight"] = torch.cat([Wk, Wv], dim=0)
            state_dict[f"encoder.layers.{d}.mixer.Wq.bias"] = bq
            state_dict[f"encoder.layers.{d}.mixer.Wkv.bias"] = torch.cat([bk, bv], dim=0)

    def key_mapping_attn(key):
        return re.sub(
            r"^encoder.layers.(\d+).attention.output.dense.(weight|bias)",
            r"encoder.layers.\1.mixer.out_proj.\2",
            key,
        )

    state_dict = OrderedDict((key_mapping_attn(k), v) for k, v in state_dict.items())

    def key_mapping_decoder_bias(key):
        return re.sub(r"^cls.predictions.bias", "cls.predictions.decoder.bias", key)

    state_dict = OrderedDict((key_mapping_decoder_bias(k), v) for k, v in state_dict.items())

    # Word embedding
    pad_vocab_size_multiple = getattr(config, "pad_vocab_size_multiple", 1)
    if pad_vocab_size_multiple > 1:
        word_embeddings = state_dict["embeddings.word_embeddings.weight"]
        state_dict["embeddings.word_embeddings.weight"] = F.pad(
            word_embeddings, (0, 0, 0, config.vocab_size - word_embeddings.shape[0])
        )
        decoder_weight = state_dict["cls.predictions.decoder.weight"]
        state_dict["cls.predictions.decoder.weight"] = F.pad(
            decoder_weight, (0, 0, 0, config.vocab_size - decoder_weight.shape[0])
        )
        # # If the vocab was padded, we want to set the decoder bias for those padded indices to be
        # # strongly negative (i.e. the decoder shouldn't predict those indices).
        # # TD [2022-05-09]: I don't think it affects the MLPerf training.
        # decoder_bias = state_dict["cls.predictions.decoder.bias"]
        # state_dict["cls.predictions.decoder.bias"] = F.pad(
        #     decoder_bias, (0, config.vocab_size - decoder_bias.shape[0]), value=-100.0
        # )

    return state_dict
state_dict_remap = remap_state_dict(bertmodel.state_dict(), config)

In [5]:
for k, v in state_dict_remap.items():
	state_dict_remap[k] = v.to(dtype=torch.float16)
state_dict_remap

OrderedDict([('embeddings.word_embeddings.weight',
              tensor([[-0.0102, -0.0616, -0.0265,  ..., -0.0199, -0.0372, -0.0098],
                      [-0.0117, -0.0600, -0.0323,  ..., -0.0168, -0.0401, -0.0107],
                      [-0.0198, -0.0627, -0.0326,  ..., -0.0165, -0.0420, -0.0032],
                      ...,
                      [-0.0218, -0.0556, -0.0135,  ..., -0.0043, -0.0151, -0.0249],
                      [-0.0462, -0.0565, -0.0019,  ...,  0.0157, -0.0139, -0.0095],
                      [ 0.0015, -0.0821, -0.0160,  ..., -0.0081, -0.0475,  0.0753]],
                     dtype=torch.float16)),
             ('embeddings.position_embeddings.weight',
              tensor([[ 1.7502e-02, -2.5635e-02, -3.6652e-02,  ...,  3.3438e-05,
                        6.8331e-04,  1.5442e-02],
                      [ 7.7591e-03,  2.2621e-03, -1.9440e-02,  ...,  2.8915e-02,
                        2.9755e-02, -5.3253e-03],
                      [-1.1284e-02, -1.9646e-03, -1.1574

In [6]:
from berttest import BertModel as FlashBertModel
fbert = FlashBertModel(config=config)
fbert.load_state_dict(state_dict_remap)

<All keys matched successfully>

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
input_ids, token_type_ids, attention_mask = tokenizer("""
In Appendix D, we examine key design choices regarding
initialization, alternating optimization, the impact of the
fine-tuning protocol, distribution of codebooks vs groups, as
well as other hyper-parameters. In brief, we first find that the
residual K-means initialization is critical for fast algorithm
convergence: when compared with random initialization,
it needs significantly fewer training iterations. Second, to
validate our calibration fine-tuning procedure, we compare
it against 1) no fine-tuning, 2) fine-tuning only of non-linear
layers (e.g. RMSNorm) but not of codebook parameters,
and 3) fine-tuning only the codebooks (but not other layers).
The results, presented in full in Appendix D, show that fine-
tuning the codebook parameters has the highest impact on
accuracy, by far, while fine-tuning the RMSNorm only has
minor impact. This validates our choice of leveraging the
calibration set for learned codebooks.
Further, we observe that increasing the number of sample
sequences in the range 128 to 4096 leads to a gradual PPL
improvement, but with diminishing returns. In this respect,
AQLM benefits more from larger calibration sets (similarly
to QuIP#), as opposed to direct methods like GPTQ which
saturate accuracy at around 256 input sequences. Finally,
we investigate various options for investing a given bit bud-
get, comparing e.g. longer codes (e.g. 1x15) vs multiple
codebooks with shorter codes (e.g. 2x8).
""").values()
position_ids = list(range(len(input_ids)))


In [8]:
fbert(input_ids=torch.tensor([input_ids], dtype=torch.long), position_ids=torch.tensor([position_ids], dtype=torch.long), token_type_ids=torch.tensor([token_type_ids], dtype=torch.long), attention_mask=torch.tensor([attention_mask], dtype=torch.long))

AssertionError: 